In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import asyncio
import aiohttp
import nest_asyncio
from selenium import webdriver
from loguru import logger
from datetime import datetime

from fake_useragent import UserAgent


nest_asyncio.apply()


## Спарсим рамблер

Это работает так: сначала код запрашивает требуемую страинцу у рамблера, тот в ответ отправляет html страницу, а затем идет поиск тега, в котором находится текст гороскопа. 

In [ ]:
signs=['aries','gemini','taurus','cancer','leo','virgo','libra','scorpio','sagittarius','capricorn','aquarius','pisces']
data = pd.read_csv('data.csv', names=['date', 'sign', 'text'])

for i in range(2004,2021):
    for j in range(1,13):
        for k in range(1,32):
            for sign in signs:
                try:
                    year=i
                    month=j
                    day=k
                    if month<10:
                        month='0'+str(month)
                    if day<10:
                        day='0'+str(day)
                    date='%s-%s-%s'%(year,month,day)
                    url = 'https://horoscopes.rambler.ru/%s/%s/?updated'% (sign,date)  # url для второй страницы
                    print(url)
                    r = requests.get(url)
                    response = r.text.encode('utf-8')

                    soup = BeautifulSoup(response, features="lxml")
                    text = soup.find('div', {'class': '_1dQ3'}).text
                    data=data.append({'date': date, 'sign': sign, 'text': text}, ignore_index=True)
                    data.to_csv('data.csv', encoding = 'utf-8')
                    print('saved date=', date, ' sign=', sign)
                except Exception:
                    continue

## Сайт Ростова-на-Дону

https://www.1rnd.ru/

In [ ]:
news=[]
url='https://www.1rnd.ru/news/3089599'
print(url)
r = requests.get(url)
response = r.text
soup = BeautifulSoup(response, 'html.parser')

news_text=soup.findAll('p', {'style': "text-align:justify;"})
date_text=soup.findAll('div', {'class': "article-info__time"})
header_text=soup.findAll('div',{'class':'title-container inner-title'})

if news_text:
    print('success')
    text_line=''
    for text in news_text:
         text_line=text_line+text.get_text()
    news.append([header_text[0].get_text(),date_text[0].get_text().split(),text_line])

In [ ]:
news = pd.DataFrame({'header': [0], 'date': [0], 'news': [0]})

In [ ]:
start1=7135
start2=300000
start3=600000
start4=900000
start5=1200000
start6=1500000
start7=1800000
start8=2100000
start9=2400000
start10=2700000

end1=299999
end2=599999
end3=899999
end4=1199999
end5=1499999
end6=1799999
end7=2099999
end8=2399999
end9=2699999
end10=2999999


async def get_news(start,end):
    global news
    for i in range(start,end):
        url ='https://www.1rnd.ru/news/{0}'.format(str(i)) 
        print(url)
        
        async with aiohttp.ClientSession() as session:
            async with session.get(url) as response:

                response = await response.text()
                soup = BeautifulSoup(response, 'html.parser')

                news_text=soup.findAll('p', {'style': "text-align:justify;"})
                date_text=soup.findAll('div', {'class': "article-info__time"})
                header_text=soup.findAll('div',{'class':'title-container inner-title'})

                if news_text:
                    print('success')
                    text_line=''
                    for text in news_text:
                         text_line=text_line+text.get_text()
                            
                    date=date_text[0].get_text()
                    header=header_text[0].get_text()

                    news=news.append({'header': header, 'date': date, 'news': text_line}, ignore_index=True)
                    news.to_csv('news.csv', encoding = 'utf-8')

futures = [get_news(start1,end1),
           get_news(start2,end2),
           get_news(start3,end3),
           get_news(start4,end4),
           get_news(start5,end5),
           get_news(start6,end6),
           get_news(start7,end7),
           get_news(start8,end8),
           get_news(start9,end9),
           get_news(start10,end10)]

loop = asyncio.get_event_loop()
loop.run_until_complete(asyncio.wait(futures))
loop.close()

Товарная единица - ивент

Нужно определить - 
 - [x] категория
 - [x] стоимость 
 - [x] геопозиция
 - [x] дата
 - [x] длительность
 - [x] количество людей
 - [ ] список услуг (недоступно)
 - [x] кто проводит
 - [x] сайт

# BookYogaRetreats

In [52]:
page=1
base_url='www.bookyogaretreats.com'
url = f'https://{base_url}/?page={page}'
print(url)
r = requests.get(url)
response = r.text.encode('utf-8')

soup = BeautifulSoup(response)
text = soup.findAll('a', {'class': 'js-showcard-link'},href=True)
card_urls=[]
for t in text:
    card_urls.append(t['href'])
# data=data.append({'date': date, 'sign': sign, 'text': text}, ignore_index=True)
# data.to_csv('data.csv', encoding = 'utf-8')
# print('saved date=', date, ' sign=', sign)

https://www.bookyogaretreats.com/?page=1


In [53]:
card_urls

['/luxury-executive-wellness/6-day-luxury-wellness-yoga-retreat-in-santa-barbara-florida',
 '/luxury-executive-wellness/6-day-holistic-wellness-retreat-in-clearwater-beach-florida',
 '/enliven-retreats/5-day-flourishing-in-uncertain-times-wellness-retreat-in-vermont',
 '/divinitree-yoga/8-day-ayurvedic-alchemy-cleanse-waterfall-and-beach-journeys-and-yoga-holiday-in-maui-hawaii',
 '/ccs-hideaway/8-day-phuket-sandbox-all-inclusive-yoga-holiday-in-karon',
 '/himalayan-yoga-association/6-day-relaxing-and-rejuvenating-yoga-retreat-in-rishikesh-india',
 '/vivir-by-v-/3-day-wellness-revive-yoga-and-fitness-holiday-in-vejer-de-la-frontera',
 '/peace-retreat-costa-rica/5-days-adventure-and-yoga-retreat-in-costa-rica',
 '/peace-retreat-costa-rica/5-day-rejuvenating-body-at-peace-yoga-wellness-retreat-in-los-pargos-guanacaste',
 '/beyond-yoga-retreat/8-day-all-inclusive-yoga-massage-meditation-and-hiking-retreat-in-puerto-vallarta-mexico',
 '/fykiada-retreats/6-day-luxury-ashtanga-yoga-retreat-w

In [58]:
page=1
base_url='www.bookyogaretreats.com'
url = f'https://{base_url}{card_urls[10]}'
# url='https://www.bookyogaretreats.com/yoga-weeks/6-day-urban-yoga-retreat-in-barcelona-catalonia'
print(url)
r = requests.get(url)
response = r.text.encode('utf-8')

soup = BeautifulSoup(response,'html.parser')
text = soup.findAll('a', {'class': 'js-showcard-link'})

https://www.bookyogaretreats.com/fykiada-retreats/6-day-luxury-ashtanga-yoga-retreat-with-laruga-glaser-and-david-fredriksson-in-mykonos


In [235]:
# soup

In [233]:
soup.findAll('div', {'id': 'error-page'})[0].text

"\n\n\n\n\n\nOops! 404 Page Not Found\nWe can’t find the page you're looking for, but we can help you find an unforgettable holiday!\n\n\nWe can help you find your way via our customer service pageOr, you can locate a specific page via our site map\n\n\nGo back\nGo to homepage\n\n\nCopyright © 2022 \nAll rights reserved.\nMade with love by\nTripaneer.\n"

In [237]:
soup.findAll('div', {'id': 'error-page'})

[]

# Completed


In [59]:
duration=soup.findAll('div', {'class': 'subtitle-small listing-query-box__duration js-sc3-inquiry-modal-duration js-insert-duration'})[0].text
duration=duration.replace('\n','').split(' ')[0]
duration

'6'

In [60]:
title=soup.findAll('h1', {'class': 'listing-title__title title listing-title-new'})[0].text
title

'6 Day Luxury Ashtanga Yoga Retreat with Laruga Glaser and David Fredriksson in Mykonos'

In [110]:
arrival=soup.findAll('span', {'class': 'js-sc1-inquiry-modal-arrival'})
if len(arrival)!=0:
    arrival=arrival[0].text
    arrival=arrival.replace(',','').split(' ')
    arrival[1]=arrival[1][:3]
    arrival=' '.join(arrival[1:])
    arrival=datetime.strptime(arrival, '%b %d %Y').date()
else:
    arrival=None
    

print(arrival)

2022-03-17


In [108]:
departure=soup.findAll('span', {'class': 'js-sc1-inquiry-modal-departure'})
if len(departure)!=0:
    departure=departure[0].text
    departure=departure.replace(',','').split(' ')
    departure[1]=departure[1][:3]
    departure=' '.join(departure[1:])
    departure=datetime.strptime(departure, '%b %d %Y').date()
else:
    departure=None


print(departure)

Mar 27 2022
2022-03-27


In [103]:
datetime.strptime(' 17 2022', '%b %d %Y').date()

datetime.date(2022, 4, 17)

In [315]:
organizer=soup.findAll('div', {'class': 'listing-organizer-card__content'})[0].text[20:].split('\n')[0]
organizer

'Yoga Weeks'

In [316]:
arr=soup.findAll('div', {'class': 'listing-overview__notes'})[0].text.split('\n')
arr=[ a for a in arr if a!='']
duration=arr[0].split(' ')[0]
duration

'6'

In [317]:
location=soup.findAll('div', {'class': 'listing-title__location listing-location-new'})
if len(location[0])!=0:
    location=location[0].text
else:
    location=None

print(location)

Call Carme 40, Pral 2, 08001, Barcelona, Spain


In [318]:
arr=soup.findAll('div', {'class': 'listing-overview__notes'})[0].text.split('\n')
arr=[ a for a in arr if a!='']
group_size=[a for a in arr if 'Group' in a]

if len(group_size)!=0:
    if len(group_size[0].split(' '))==6:
         group_size=group_size[0].split(' ')[4]
    elif len(group_size[0].split(' '))==4:
        group_size=group_size[0].split(' ')[2]
else:
    group_size=None

print(arr)
print(group_size)

['6 days with instruction in\xa0English', 'The organizer also speaks:\xa0French, German, Spanish', 'Group size: Maximum of 10 participants']
10


In [ ]:
driver = webdriver.Firefox(executable_path= r"D:\загрузки\geckodriver-v0.30.0-win64\geckodriver.exe")

In [252]:
driver.get(url)
driver.find_elements_by_class_name('price')[0].text.split(' ')[1]

<ipython-input-252-a47f4b609ba8>:2: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  driver.find_elements_by_class_name('price')[0].text.split(' ')[1]


'97'

In [300]:
#main page

main_page_url='https://www.bookyogaretreats.com'
r = requests.get(main_page_url)
response = r.text.encode('utf-8')

soup = BeautifulSoup(response)

categories_raw=soup.findAll('li', {'class': 'fakeCheck'})
categories_raw=[' '.join(category.text.replace('\n','').split(' ')[:-1]) for category in categories_raw][12:-4]
categories=[]

for i,category in enumerate(categories_raw):
        if 'level' in category.lower():
            categories.append(category.split(' ')[1])
        else:
            categories.append(category)

# categories

In [238]:
soup.findAll('div', {'id': 'error-page'})

[]

# Pipeline

In [257]:
'https://www.bookyogaretreats.com/sr?c=budget-retreats&page=1'

In [2]:
data = pd.DataFrame({'title': [0],'duration': [0], 'arrival': [0], 'departure': [0], 'organizer': [0], 'location': [0], 'group_size': [0], 'price': [0]})
# data_categories={}

In [4]:
main_page_url='https://www.bookyogaretreats.com'

ua = UserAgent(verify_ssl=False)
headers = {'User-Agent': ua.random}


r = requests.get(main_page_url,headers=headers)
response = r.text.encode('utf-8')
soup = BeautifulSoup(response)

categories_raw=soup.findAll('li', {'class': 'fakeCheck'})
categories_raw=[' '.join(category.text.replace('\n','').split(' ')[:-1]) for category in categories_raw][12:-4]
categories=[]

driver = webdriver.Firefox(executable_path= r"D:\загрузки\geckodriver-v0.30.0-win64\geckodriver.exe")

for i,category in enumerate(categories_raw):
        if 'level' in category.lower():
            categories.append(category.split(' ')[1])
        else:
            categories.append(category)


# c_u=category_page_url=main_page_url+f'/sr?c={cat}&page={page}'
            
for page in range(147,428):
#     cat='-'.join(category.split(' ')).lower()
    cat='cat '
#     stop=False

#     data_categories[cat]=[]
#     while stop is False:
        
    page_url=main_page_url+f'/?page={page}'
    print('NEW Page ',page_url,'\n----------------------')

    # страница каталога 
    r = requests.get(page_url,headers=headers)
    response = r.text.encode('utf-8')
    soup = BeautifulSoup(response)

    # если такой страницы нет, т.е. каталог закончился
    error=soup.findAll('div', {'id': 'error-page'})

    if len(error)>0:
        stop=True
        print(f'####Category page {page} ERROR####')
        break

    text = soup.findAll('a', {'class': 'js-showcard-link'},href=True)
    card_urls=[]
    for t in text:
        card_urls.append(t['href'])

    for card_url in card_urls:

        url = f'{main_page_url}{card_url}'
        print(f'category {cat}, url {url}')
        r = requests.get(url,headers=headers)
        response = r.text.encode('utf-8')
        soup = BeautifulSoup(response,'html.parser')

        error=soup.findAll('div', {'id': 'error-page'})
        if len(error)>0:
            continue


        arr=soup.findAll('div', {'class': 'listing-overview__notes'})[0].text.split('\n')
        arr=[ a for a in arr if a!='']
        duration=arr[0].split(' ')[0]

        title=soup.findAll('h1', {'class': 'listing-title__title title listing-title-new'})[0].text
#             data_categories[cat].append(title)

        arrival=soup.findAll('span', {'class': 'js-sc1-inquiry-modal-arrival'})
        if len(arrival)!=0:
            arrival=arrival[0].text
            arrival=arrival.replace(',','').split(' ')
            arrival[1]=arrival[1][:3]
            arrival=' '.join(arrival[1:])
            arrival=datetime.strptime(arrival, '%b %d %Y').date()
        else:
            arrival=None

        departure=soup.findAll('span', {'class': 'js-sc1-inquiry-modal-departure'})
        if len(departure)!=0:
            departure=departure[0].text
            departure=departure.replace(',','').split(' ')
            departure[1]=departure[1][:3]
            departure=' '.join(departure[1:])
            departure=datetime.strptime(departure, '%b %d %Y').date()
        else:
            departure=None

        organizer=soup.findAll('div', {'class': 'listing-organizer-card__content'})[0].text[20:].split('\n')[0]

        location=soup.findAll('div', {'class': 'listing-title__location listing-location-new'})
        if len(location[0])!=0:
            location=location[0].text
        else:
            location=None

        arr=soup.findAll('div', {'class': 'listing-overview__notes'})[0].text.split('\n')
        arr=[ a for a in arr if a!='']
        group_size=[a for a in arr if 'Group' in a]

        if len(group_size)!=0:
            if len(group_size[0].split(' '))==6:
                 group_size=group_size[0].split(' ')[4]
            elif len(group_size[0].split(' '))==4:
                group_size=group_size[0].split(' ')[2]
        else:
            group_size=None

        driver.get(url)
        
        price_list=driver.find_elements_by_class_name('price')[0].text.split(' ')
        if len(price_list)<2:
            price=None
        else:
            price=price_list[1]

        data=data.append({'title': [title],'duration': [duration], 'arrival': [arrival], 'departure':[departure] ,
                          'organizer': [organizer], 'location': [location], 'group_size': [group_size],
                          'price': [price]}, ignore_index=True)
    data.to_csv('data.csv', encoding = 'utf-8')
#         np.save('data_categories.npy',data_categories)

#         page+=1

<ipython-input-4-2945dbf657e9>:15: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Firefox(executable_path= r"D:\загрузки\geckodriver-v0.30.0-win64\geckodriver.exe")


NEW Page  https://www.bookyogaretreats.com/?page=137 
----------------------
category cat , url https://www.bookyogaretreats.com/sanatana-yog-sandesh/10-days-dawn-into-yoga-retreat-in-rishikesh-india


<ipython-input-4-2945dbf657e9>:117: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  price_list=driver.find_elements_by_class_name('price')[0].text.split(' ')


category cat , url https://www.bookyogaretreats.com/peaceful-mind-the-ancient-yoga/7-day-meditation-and-kundalini-yoga-online-retreats
category cat , url https://www.bookyogaretreats.com/my-holistic-bliss-ltd/3-days-yin-yoga-acupuncture-and-wellbeing-retreat-in-hutton-le-hole-north-yorkshire
category cat , url https://www.bookyogaretreats.com/sanatana-yog-sandesh/21-day-200-hour-online-hatha-yoga-teacher-training-course
category cat , url https://www.bookyogaretreats.com/kukui-surf-yoga/6-day-flowing-desert-yoga-and-surf-holiday-in-imsouane
category cat , url https://www.bookyogaretreats.com/yoga-shakti-school/6-month-100-hour-interactive-online-one-to-one-healing-yoga-teacher-training
category cat , url https://www.bookyogaretreats.com/kausay/5-day-grounding-and-nourishing-silent-walking-retreat-in-nature-with-daily-yoga-in-valencia
category cat , url https://www.bookyogaretreats.com/bala-yoga-school/4-week-300-hour-advanced-online-hatha-and-vinyasa-yoga-teacher-training-course
catego

category cat , url https://www.bookyogaretreats.com/pathways-country-yoga-retreat/6-day-or-self-paced-online-yoga-nidra-self-help-course
category cat , url https://www.bookyogaretreats.com/yoga-farm-ithaca/7-month-spiritual-warrior-300-hour-online-kundalini-yoga-teacher-training-with-6-week-winter-break
NEW Page  https://www.bookyogaretreats.com/?page=142 
----------------------
category cat , url https://www.bookyogaretreats.com/nirvana-yoga-shala/61-day-200-hour-online-kundalini-yoga-teacher-training-international-certification-yoga-alliance
category cat , url https://www.bookyogaretreats.com/shanti-yoga-ashram/online-self-paced-100-hour-tantra-yoga-course
category cat , url https://www.bookyogaretreats.com/yoga-maithri/5-days-nada-mantra-yoga-retreat-in-kerala-india
category cat , url https://www.bookyogaretreats.com/naya-ubud/5-days-balinese-culture-and-yoga-retreat-in-bali-indonesia
category cat , url https://www.bookyogaretreats.com/atmashree-yoga-retreat/4-day-short-and-affordab

category cat , url https://www.bookyogaretreats.com/ajo-wellness-group/3-day-thrive-women-s-wellbeing-retreat-with-yoga-and-mindfulness-in-magaliesburg
category cat , url https://www.bookyogaretreats.com/soulflow-oakland/5-day-awakening-your-full-potential-mount-shasta-meditation-retreat-with-yoga-and-martial-arts
category cat , url https://www.bookyogaretreats.com/meditation-and-yoga-school/4-days-3-nights-healing-silent-meditation-nature-retreat-near-the-appalachian-mountains
category cat , url https://www.bookyogaretreats.com/nirvana-yoga-and-meditation/5-day-50-hour-online-ashtanga-yoga-teacher-training-course
category cat , url https://www.bookyogaretreats.com/yogada-in-the-himalayas/7-day-atam-sukha-wellness-retreat-with-yoga-and-ayurveda-in-shivpuri-uttarakhand
category cat , url https://www.bookyogaretreats.com/santosa-detox-and-wellness-center/7-days-raw-food-fast-and-detox-retreat-in-phuket-thailand
category cat , url https://www.bookyogaretreats.com/elephant-pass-ayurveda-yo

UnexpectedAlertPresentException: Alert Text: None
Message: Dismissed user prompt dialog: Не удается связаться с сервисом reCAPTCHA. Проверьте подключение к Интернету и повторите попытку.
Stacktrace:
WebDriverError@chrome://marionette/content/error.js:175:5
UnexpectedAlertOpenError@chrome://marionette/content/error.js:460:5
GeckoDriver.prototype._handleUserPrompts@chrome://marionette/content/driver.js:3374:13


In [33]:
driver.find_elements_by_class_name('price')[0].text

<ipython-input-33-8ea8e88faf2c>:1: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  driver.find_elements_by_class_name('price')[0].text


'--'

In [258]:
cat='-'.join(categories[0].split(' '))
category_page_url=main_page_url+f'/sr?c={cat}'